<a href="https://colab.research.google.com/github/MattWroclaw/data-science-bootcamp/blob/main/Uczenie_maszynowe/03_metryki_regresja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Metody oceny modeli regresyjnych

*Model regresyjny* (przewidzenie ciągłej zmiennej) : ma on za zadanie przewidzieć ciągłą wartość naszej zmiennej. Np. cena akcji na giełdzie, liczba klientów w danym dni, miesiącu..

## Metryki Problem regresji


1.  Import blibliotek `pip install scikit-learn`
2.  Interpretacja graficzna
3.  [Mean Absolute Error - MAE - Średni bład bezwzględny](#a3)
4.  [Mean Squared Error - MSE - Błądśredniokwadratowy](#a4)
5.  [Root Mean Squared Error - RMSE - pierwiastek błędu śreniokwadratowego](#a5)
6.  [Max Error - Błąd maksymalny](#a6)
7.  [R2 soruce -współczynnik determinacji](#a7)




In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# generujemy dane "prawdziwe"
y_true = 100 + 20 * np.random.randn(50)
y_true

array([ 51.86865032, 139.28719827,  60.4063477 ,  79.28367365,
       133.32081446, 120.43436484, 131.23733011,  67.68234215,
        69.7839212 ,  94.22032113,  63.01227304, 110.91164086,
       110.2693364 ,  94.28049412, 119.9893985 ,  91.59382815,
        95.09721001, 108.15699432,  68.81827974,  91.32308968,
       124.36710229, 104.66986408,  97.95164432,  71.94634939,
       132.53053872, 136.12763626, 116.49762433, 109.75481596,
       118.03362514, 109.8555594 ,  96.75254431,  93.12432197,
       109.39475373, 131.4053201 , 114.9301534 ,  97.22205547,
       117.07552472,  96.97347284,  97.8073768 , 122.52634898,
        98.32649432, 112.55275311, 127.78568002, 100.05969652,
       104.98410333,  88.89042711,  78.76056765,  78.3346076 ,
       114.37642314,  56.09995338])

In [3]:
# generujemy dane przewidywane, zależe + szum (50 -> 50 liczb)

y_pred = y_true + 10 * np.random.randn(50)
y_pred

array([ 46.25664562, 130.71602655,  62.01797877,  66.6398914 ,
       133.63228726, 118.31831661, 141.96519622,  53.42495959,
        70.62864606, 103.12028718,  49.10179232, 121.27509625,
       112.7789572 , 107.05621175, 116.30839489, 102.27980777,
       106.75112044, 115.60496493,  83.4705005 ,  90.35747897,
       128.6159185 , 101.46042476, 102.68385983,  57.53261697,
       120.69651411, 118.46098488, 124.19910994, 121.47117188,
       111.00699709,  95.09864994,  88.80128415,  90.82034894,
       101.48200358, 124.70496001, 120.66470008,  97.75654248,
       118.4546571 , 102.37989408,  81.85638312, 128.84296513,
        85.62158562, 114.91043561, 123.55497429, 101.91678551,
       109.09691766,  96.37477955,  71.33471658,  74.28827568,
       119.34248428,  46.00990499])

In [6]:
# Tworzymy ramke danych, która pozwoli nam pokazać graficznie wykres danych

results = pd.DataFrame({'y_true': y_true, 'y_pred':y_pred})
results.head()

,y_true,y_pred
0,51.868650,46.256646
1,139.287198,130.716027
2,60.406348,62.017979
3,79.283674,66.639891
4,133.320814,133.632287


In [7]:
# zbadamy błąd, czyli różnicę prawda - wartość przwidziana

results['error'] = results['y_true'] - results['y_pred']
results.head()

,y_true,y_pred,error
0,51.868650,46.256646,5.612005
1,139.287198,130.716027,8.571172
2,60.406348,62.017979,-1.611631
3,79.283674,66.639891,12.643782
4,133.320814,133.632287,-0.311473


# Interpretacja graficzna

In [9]:
def plot_regression_results(y_true, y_pred):
  results = pd.DataFrame({'y_true': y_true, 'y_pred':y_pred})
  min = results[['y_true', 'y_pred']].min().min()
  max = results[['y_true', 'y_pred']].max().max()

  fig = go.Figure(data=[go.Scatter(x=results['y_true'], y=results['y_pred'], mode='markers' ),
                        go.Scatter(x=[min, max], y=[min, max])], #to jest ten czerwony ślad
                  layout=go.Layout(width=800,
                                  xaxis_title='y_true',
                                  yaxis_title='y_pred',
                                  title='Regression results'))
  fig.show()

plot_regression_results(y_true, y_pred)

In [12]:
#  teraz będziemy badać histogram błędów (czyli rozkład)
#  Potrzebujemy więcej danych

y_true = 100 + 20 *np.random.randn(1000)
y_pred = y_true + 10 * np.random.randn(1000)

results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results['error'] = results['y_true'] - results['y_pred']

px.histogram(results, x='error', nbins=50, width=800)

Zazwyczaj powinien przypominać rozkład normalny. Gdy jest lewo / prawo skośny to oznacza, że przeszacowuje albo niedoszacowuje. Jak jest więcej błędów po prawej stronie to ozacza, że model niedoszacowuje wartości.  
Gdy jest więcej błędów po lewej stronie to oznacza ze model przeszacowuje wartości.  
*Optymalnie* gdy jest symetryczny.
Gdy histogram jest *rozwlekły* to oznacza że model tworzy duże błędy.  


## <a name='a3'></a> Mean Absolute Error - Średni błąd bezwzględny
Standardowa metryka.

### $$MAE = \frac{1}{n}\sum_{i=1}^{n}|y_{true} - y_{pred}|$$

In [14]:
def mean_absolute_error(y_true, y_pred):
  return abs(y_true - y_pred).sum() / len(y_true)

mean_absolute_error(y_true, y_pred)

7.882351823561818

In [15]:
# Zamiast tego możemy korzystać z gotowej metody

from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_true, y_pred)

7.882351823561818

### <a name='a4'></a> Mean Squared Error - MSE - Błąd średniokwadratowy
### $$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_{true} - y_{pred})^{2}$$

Błąd będzie karał większe odległości.

In [16]:
def mean_squared_error(y_true, y_pred):
  return ((y_true-y_pred)**2).sum() / len(y_true)

mean_squared_error(y_true, y_pred)

98.85967855930728

In [17]:
# To samo dzięki wbudowanej bibliotece
from sklearn.metrics import mean_squared_error
mean_squared_error(y_true, y_pred)

98.85967855930728

### <a name='a5'></a> Root Mean Squared Error - RMSE - Pierwiastek błędu średniokwadratowego
### $$RMSE = \sqrt{MSE}$$

Ta metryka pozwala nieco lepiej zinterpretować nasz wynik.  
Wynik mówi nam jak średnio predykcje odbiegają od wartości prawdziwych.

In [18]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(((y_true - y_pred) ** 2).sum() / len(y_true))

root_mean_squared_error(y_true, y_pred)

9.942820452935237

In [19]:
np.sqrt(mean_squared_error(y_true, y_pred))

9.942820452935237

Nasze wartości odbiegają średnio o 9.9428 (np. zł)

### <a name='a6'></a>  Max Error - Błąd maksymalny

$$ME = max(|y\_true - y\_pred|)$$

In [21]:
def max_error(y_true, y_pred):
    return abs(y_true - y_pred).max()

max_error(y_true, y_pred)

38.86190526931087

In [22]:
# gotowa metoda

from sklearn.metrics import max_error

max_error(y_true, y_pred)

38.86190526931087

### <a name='a7'></a>  R2 score - współczynnik determinacji
### $$R2\_score = 1 - \frac{\sum_{i=1}^{N}(y_{true} - y_{pred})^{2}}{\sum_{i=1}^{N}(y_{true} - \overline{y_{true}})^{2}}$$

To w mianowniku to jest średnia."Jak byśmy wzięli model, który dla każdego przypadku będzie przewidywał średnią wartość , i jak nasz model działa lepiej poza ten zwykly model"  
Chodzi o to, żeby ta wartość była jak najbliżej 1.  
Jest to domyślna metoda, która sotsuje biblioteka sklearn do ewaluacji modeli regresji.

In [24]:


from sklearn.metrics import r2_score

r2_score(y_true, y_pred)


0.7396561155757666

In [25]:
def r2_score(y_true, y_pred):
    numerator = ((y_true - y_pred) ** 2).sum()
    denominator = ((y_true - y_true.mean()) ** 2).sum()
    try:
        r2 = 1 - numerator / denominator
    except ZeroDivisionError:
        print('Dzielenie przez zero')
    return r2

In [26]:
r2_score(y_true, y_pred)

0.7396561155757666